<a href="https://colab.research.google.com/github/ruperty/PCTMonitor/blob/master/basic_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils  > /dev/null 2>&1

Cartpole

In [0]:

import functions.test as f
x=3
print(myfunc(x))

ModuleNotFoundError: ignored

In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start() 

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                
                 controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
env = wrap_env(gym.make("CartPole-v1"))
state = env.reset()
#print(observation)
loops = 100
plot_data = [[0],[state[0]],[state[1]],[state[2]],[state[3]], []]
for i in range(loops):
   env.render()
   action = env.action_space.sample()
   observation =env.step(action) # take a random action
   #print(action, observation)
   state = observation[0]
   plot_data[0].append(i+1)
   plot_data[1].append(state[0])
   plot_data[2].append(state[1])
   plot_data[3].append(state[2])
   plot_data[4].append(state[3])
   if action : dir=1 
   else: dir=-1
   plot_data[5].append(dir)

   done = observation[2]
   if done: 
        if i < loops :
          print("Failed after %d loops with a pole angle of %2.2f degrees" %( i, np.rad2deg(observation[0][2])))
        else:
          print("Finished after ", i, " loops")
        env.reset()
        break

env.close()
show_video()
#print(plot_data)

Failed after 18 loops with a pole angle of 12.11 degrees


In [0]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[1], name="cartPos"))
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[2], name="cartVel"))
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[3], name="poleAng"))
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[4], name="poleVel"))
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[5], name="direction"))


fig.show()

In [0]:
def proportional(r, p, g):
    return (r-p)*g

def controller(ctr,     pole_angle_ref, cart_position, cart_velocity, pole_angle, pole_velocity):
    
    pole_velocity_ref=proportional(pole_angle_ref, pole_angle, pole_angle_gain)
    cart_position_ref=proportional(pole_velocity_ref, pole_velocity, pole_velocity_gain)
    cart_velocity_ref=proportional(cart_position_ref, cart_position, cart_position_gain)
    power=proportional(cart_velocity_ref, cart_velocity, cart_velocity_gain)
    
    if power>=0:
        return 0
    return 1

def get_obs(observation):
    cart_position = observation[0]
    cart_velocity = observation[1]
    pole_angle = observation[2]
    pole_velocity = observation[3]
    return cart_position,    cart_velocity ,    pole_angle , pole_velocity 


In [0]:
pole_angle_ref = 0

loops = 500
pole_angle_gain = 1.8
pole_velocity_gain = 1.8
cart_position_gain = .5
cart_velocity_gain = .001

In [0]:
genv = gym.make("CartPole-v1")
genv._max_episode_steps = 4000
env = wrap_env(genv)
reward = 0

observation = env.reset()
cart_position = observation[0]
cart_velocity = observation[1]
pole_angle = observation[2]
pole_velocity = observation[3]
for ctr in range(loops):
    env.render()
    #action= env.action_space.sample()
    #print(action)
    
    action = controller(ctr, pole_angle_ref, cart_position, cart_velocity, pole_angle, pole_velocity)
    
    observation =env.step(action)    
    #print(observation)
    done = observation[2]
    reward+=observation[1]
    cart_position, cart_velocity, pole_angle, pole_velocity = get_obs(observation[0])
    #print(cart_position)
    if done: 
        print(ctr, observation)
        break
 
print("Reward ", reward, " from ", ctr+1, " loops")    

env.close()
show_video()

Reward  500.0  from  500  loops
